# Initial data exploration
This is a simple initial data exploration created by Nick.

The objective of this notebook in particular is the following:
- Know how to read in the three different datasets that we have
    - Might keep it only to the plain text and navigation paths, HTML might be too much
- Find a way of linking the two datasets
- Enrich the graph with the shortest path, or find a way of adding that info as well
- Maybe see if there's a way of doing basic AF explorations already with networkX?

Might also be worth it to create the environment we'll use for this...

In [2]:
import numpy as np
import pandas as pd
import networkx
import matplotlib.pyplot as plt


# How to use data readers
Most of this notebook is just the template code that was used to make sure everything works.

The important part is the following code block, that shows how to read in each of the different datasets that we have.

In [3]:
import data_readers

# The links and edges
wikispeedia = data_readers.read_wikispeedia_graph()

# The finished paths
finished_paths = data_readers.read_finished_paths()

# The unfinished paths
unfinished_paths = data_readers.read_unfinished_paths()

# The shortest path matrix
# This one is the slowest to read by far, probably due to the weird parsing that has to be done!
shortest_path_df = data_readers.read_shortest_path_df()

# Searching for the string of a given article. It has to be formatted like the article name
# Which shouldn't be a problem, as we'll probably usually retrieve them internally
obi_wan_text = data_readers.plaintext_article_finder('Obi-Wan_Kenobi')

In [ ]:
shortest_path_df[('Actor',)][('Japan',)]

In [ ]:
shortest_path_df

In [ ]:
shortest_path_df

,"(%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,)","(%C3%85land,)","(%C3%89douard_Manet,)","(%C3%89ire,)","(%C3%93engus_I_of_the_Picts,)","(%E2%82%AC2_commemorative_coins,)","(10th_century,)","(11th_century,)","(12th_century,)","(13th_century,)",...,"(Ziad_Jarrah,)","(Zimbabwe,)","(Zinc,)","(Zinc_chloride,)","(Zion_National_Park,)","(Zionism,)","(Zirconium,)","(Zoroaster,)","(Zuid-Gelders,)","(Zulu,)"
"(%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,)",0,-1,-1,-1,-1,-1,3,3,3,3,...,4,3,3,4,4,3,4,4,4,2
"(%C3%85land,)",-1,0,-1,-1,-1,-1,2,2,2,2,...,4,2,3,4,4,3,4,3,3,3
"(%C3%89douard_Manet,)",-1,-1,0,-1,-1,-1,3,3,2,2,...,4,3,2,3,4,3,4,3,3,3
"(%C3%89ire,)",-1,-1,-1,0,-1,-1,3,3,3,3,...,4,2,2,3,4,3,4,4,3,3
"(%C3%93engus_I_of_the_Picts,)",-1,-1,-1,-1,0,-1,2,2,3,2,...,4,2,3,4,4,3,4,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(Zionism,)",-1,-1,-1,-1,-1,-1,2,2,2,2,...,3,2,2,3,3,0,3,3,3,2
"(Zirconium,)",-1,-1,-1,-1,-1,-1,3,3,3,3,...,3,3,2,2,3,3,0,3,4,3
"(Zoroaster,)",-1,-1,-1,-1,-1,-1,2,2,2,2,...,3,2,2,3,4,3,3,0,3,3
"(Zuid-Gelders,)",-1,-1,-1,-1,-1,-1,3,3,3,3,...,4,3,3,4,4,3,5,4,0,3


There's four really important datasets in the wikispedia articles:
- links.tsv: Contains the actual edges
- paths_finished.tsv: Contains the winning games
- paths_unfinished.tsv: Contains the losing games
- shortest-path-distance-matrix.txt: Contains info on the shortest path

First part is checking that the reading of the data is correct. We know the number of edges and nodes in the dataset, so we'll just use that to compare

In [5]:
wikispeedia= networkx.read_edgelist('datasets/wikispeedia_paths-and-graph/links.tsv', 
                                    create_using=networkx.DiGraph)
print("Dataset has", len(wikispeedia.nodes), "nodes")
print("Dataset has", len(wikispeedia.edges), "edges")

Dataset has 4592 nodes
Dataset has 119882 edges


In [ ]:
wikispeedia= networkx.read_edgelist('datasets/wikispeedia_paths-and-graph/links.tsv',
                                    create_using=networkx.DiGraph)

These are less nodes than the reported number, it should be 4,604 nodes.

The 119,882 edges is correct though.

The difference is still small-ish, so for now I'll just ignore it and focus on reading in the other datasets.

In [6]:
paths_finished = pd.read_csv('datasets/wikispeedia_paths-and-graph/paths_finished.tsv', sep='\t', skiprows=15, 
                   names=['hashedIpAddress', 'timestamp', "durationInSec", 'path', "rating"])
paths_finished.head()

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0


In [7]:
paths_unfinished = pd.read_csv('datasets/wikispeedia_paths-and-graph/paths_unfinished.tsv', sep='\t', skiprows=16,
                                names=['hashedIpAddress', 'timestamp', "durationInSec", 'path', "target", "type"])

paths_unfinished.head()

,hashedIpAddress,timestamp,durationInSec,path,target,type
0,2426091a53125110,1297054935,1804,Obi-Wan_Kenobi,Microsoft,timeout
1,26141fd878806294,1297055651,1805,Julius_Caesar,Caracas,timeout
2,2b015fb8181c48f2,1297090819,1818,Malawi;Democracy;Alexander_the_Great,First_Crusade,timeout
3,53a53bc244e08a6a,1297094761,49,Paraguay,Mount_St._Helens,restart
4,53a53bc244e08a6a,1297099105,1808,Paraguay;Bolivia,Mount_St._Helens,timeout


Last part left is the shortest distance matrix. For this part, I just need to link things up with the articles.tsv to find the names corresponding to everything.

Already reading the shortest distances is a bit of a pain though...



In [8]:
shortest_path = np.genfromtxt("datasets/wikispeedia_paths-and-graph/shortest-path-distance-matrix.txt",
                              delimiter=1, missing_values=-1, dtype=int)
articles = pd.read_csv('datasets/wikispeedia_paths-and-graph/articles.tsv', sep='\t', skiprows=12,
                       names=["article_name"])

shortest_path_df = pd.DataFrame(shortest_path, index=articles.values, columns=articles.values)

In [20]:
np.average(shortest_path_df)
mask = [shortest_path_df == -1]
# shortest_path_df[mask]

shortest_path_df_temp = shortest_path_df.replace([-1, 0], pd.NA)

# Calculate the average for each column
average_values = shortest_path_df_temp.mean()
average_values

(%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,)         NaN
(%C3%85land,)                               NaN
(%C3%89douard_Manet,)                       NaN
(%C3%89ire,)                                NaN
(%C3%93engus_I_of_the_Picts,)               NaN
                                         ...   
(Zionism,)                             2.795593
(Zirconium,)                            3.53534
(Zoroaster,)                           3.008072
(Zuid-Gelders,)                        3.364747
(Zulu,)                                2.805846
Length: 4604, dtype: object

Common file reader has already been created, I'll just have to do wait to transform things into a python file.

## Plain text reader
Another basic AF reader, the objective of this is simply to find the relevant text file given the string name. Annoying because of the format the strings were given.

In [9]:
text_file = open("datasets/plaintext_articles/%C3%85land.txt", "r", encoding="utf8")

#read whole file to a string
data = text_file.read()

#close file
text_file.close()

In [10]:
def plaintext_article_finder(article_name: str) -> str:
    art_file_name = "datasets/plaintext_articles/" + article_name + ".txt"
    text_file = open(art_file_name, "r", encoding="utf8")
    res_string = text_file.read()
    text_file.close()
    
    return res_string

# Convert URL-encoded string to a regular Python string

In [11]:
from urllib.parse import unquote

# Decodes URL-encoded article names. 
# Changes to text with accents. For example, %C3%81ed%C3%A1n_mac_Gabr%C3%A1in becomes Áedán_mac_Gabráin.
def decode_article(article_name):
    encoded_string = (article_name)
    decoded_string = unquote(encoded_string)
    return decoded_string
    
decode_article(articles.iloc[0,0])

'Áedán_mac_Gabráin'